# Limpieza de los datos de los autobuses

In [1]:
# Importamos las librerias necesarias
import pandas as pd
import os
import pyproj
import warnings
warnings.filterwarnings("ignore")

### Lineas

In [2]:
# leemos el archivo
df_lineas=pd.read_excel("fuente_datos/Lines.xlsx")
df_lineas.head()

,GroupNumber,DateFirst,DateEnd,Line,Label,NameA,NameB
0,110,22/08/2015,31/12/2999,1,1,PLAZA DE CRISTO REY,PROSPERIDAD
1,110,22/08/2015,31/12/2999,2,2,PLAZA DE MANUEL BECERRA,AVENIDA DE REINA VICTORIA
2,110,31/10/2015,31/12/2999,3,3,PUERTA DE TOLEDO,PLAZA DE SAN AMARO
3,110,30/01/2014,31/12/2999,4,4,PLAZA DE CIUDAD LINEAL,PUERTA DE ARGANDA
4,110,26/11/2014,31/12/2050,5,5,PUERTA DEL SOL/SEVILLA,ESTACION DE CHAMARTIN


In [3]:
#Vemos la forma de nuestros datos
df_lineas.shape

(209, 7)

In [4]:
# Cambiamos el nombre de las columnas
df_lineas=df_lineas.rename({"GroupNumber":"Tipo",
                            "DateFirst":"FechaInicio",
                            "DateEnd":"FechaFin",
                            "Line":"Id_Linea",
                            "Label":"Linea",
                            "NameA":"Inicio",
                            "NameB":"Fin"},
                           axis=1)
df_lineas.head()

,Tipo,FechaInicio,FechaFin,Id_Linea,Linea,Inicio,Fin
0,110,22/08/2015,31/12/2999,1,1,PLAZA DE CRISTO REY,PROSPERIDAD
1,110,22/08/2015,31/12/2999,2,2,PLAZA DE MANUEL BECERRA,AVENIDA DE REINA VICTORIA
2,110,31/10/2015,31/12/2999,3,3,PUERTA DE TOLEDO,PLAZA DE SAN AMARO
3,110,30/01/2014,31/12/2999,4,4,PLAZA DE CIUDAD LINEAL,PUERTA DE ARGANDA
4,110,26/11/2014,31/12/2050,5,5,PUERTA DEL SOL/SEVILLA,ESTACION DE CHAMARTIN


In [5]:
#Rellenamos el valor missing de la navidad
df_lineas["Fin"].fillna("BUS DE LA NAVIDAD", inplace=True)

In [6]:
#Reemplazamos la denominacion de los tipos de autobuses
df_lineas["Tipo"].replace({110:"Diario",
                        410:"Diario",
                        320:"Nocturno",
                        120:"Trabajo",
                        210:"Universitario",
                        155:"Mini",
                        171:"SE",
                        160:"SE",
                        420:"Nocturno",
                        620:"Bus Navidad"},
                       inplace=True)
df_lineas.head()

,Tipo,FechaInicio,FechaFin,Id_Linea,Linea,Inicio,Fin
0,Diario,22/08/2015,31/12/2999,1,1,PLAZA DE CRISTO REY,PROSPERIDAD
1,Diario,22/08/2015,31/12/2999,2,2,PLAZA DE MANUEL BECERRA,AVENIDA DE REINA VICTORIA
2,Diario,31/10/2015,31/12/2999,3,3,PUERTA DE TOLEDO,PLAZA DE SAN AMARO
3,Diario,30/01/2014,31/12/2999,4,4,PLAZA DE CIUDAD LINEAL,PUERTA DE ARGANDA
4,Diario,26/11/2014,31/12/2050,5,5,PUERTA DEL SOL/SEVILLA,ESTACION DE CHAMARTIN


In [7]:
# Funcion para cambiar el formato de las fechas
def formato_fecha(fecha:str)->str:
    
    dia, mes, ano=fecha.split("/")
    
    return f"{ano}-{mes}-{dia}"

df_lineas["FechaInicio"]=df_lineas["FechaInicio"].apply(formato_fecha)
df_lineas["FechaFin"]=df_lineas["FechaFin"].apply(formato_fecha)

In [8]:
# Ordenamos los datos como queremos
df_lineas=df_lineas[["Id_Linea","Linea","Inicio","Fin","Tipo"]]
df_lineas.head()

,Id_Linea,Linea,Inicio,Fin,Tipo
0,1,1,PLAZA DE CRISTO REY,PROSPERIDAD,Diario
1,2,2,PLAZA DE MANUEL BECERRA,AVENIDA DE REINA VICTORIA,Diario
2,3,3,PUERTA DE TOLEDO,PLAZA DE SAN AMARO,Diario
3,4,4,PLAZA DE CIUDAD LINEAL,PUERTA DE ARGANDA,Diario
4,5,5,PUERTA DEL SOL/SEVILLA,ESTACION DE CHAMARTIN,Diario


In [9]:
# Pasamos los datos a csv
df_lineas.to_csv("lineas.csv", index=False)

### Paradas

In [10]:
# leemos el archivo
df_paradas=pd.read_excel("fuente_datos/NodesLines.xlsx")
df_paradas.head()

,Node,PosxNode,PosyNode,Name,Lines
0,1,"433743,5",4480432,AV.VALDEMARIN-ALTAIR,161/1
1,2,"433481,3",4480271,AV.VALDEMARIN-LA SALLE,161/1
2,3,433656,4479857,BLANCA DE CASTILLA-Cº DE LA ZARZUELA,161/1
3,4,433632,4479467,PLEYADES-ANA TERESA,161/1
4,5,"433613,6",4479245,PLEYADES-AV.OSA MAYOR,161/1


In [11]:
# Limpiamos las lineas con el sentido
lista_paradas=[]

for nodo, parada in df_paradas[["Node","Lines"]].values.tolist():
    
    #Comprobamos que hay solo una linea en esa parada
    if len(parada.split(" "))==1:
        
        linea, sentido=parada.split("/")
        
        lista_paradas.append((nodo, linea, sentido))
    
    else:
        
        #Iteramos por cada linea de esa parada
        for linea in parada.split(" "):
            
            sublinea, sentido=linea.split("/")
        
            lista_paradas.append((nodo, sublinea, sentido))

df_paradas_sentido=pd.DataFrame(lista_paradas, columns=["Node","Linea","Sentido"])
df_paradas_sentido.head()

,Node,Linea,Sentido
0,1,161,1
1,2,161,1
2,3,161,1
3,4,161,1
4,5,161,1


In [12]:
#Hacemos join por el campo (Node) para juntar el dataframe original y el limpio
df_final=df_paradas_sentido.merge(df_paradas, on="Node", how="left")

In [13]:
#Reemplazamos la denominacion de los sentidos de las lineas
df_final["Sentido"].replace({"1":"Ida","2":"Vuelta"},inplace=True)

In [14]:
#Eliminamos la columna de los sentidos original
df_final=df_final.drop(["Lines"], axis=1)
df_final.head()

,Node,Linea,Sentido,PosxNode,PosyNode,Name
0,1,161,Ida,"433743,5",4480432,AV.VALDEMARIN-ALTAIR
1,2,161,Ida,"433481,3",4480271,AV.VALDEMARIN-LA SALLE
2,3,161,Ida,433656,4479857,BLANCA DE CASTILLA-Cº DE LA ZARZUELA
3,4,161,Ida,433632,4479467,PLEYADES-ANA TERESA
4,5,161,Ida,"433613,6",4479245,PLEYADES-AV.OSA MAYOR


In [15]:
#Renombramos las columnas
df_final=df_final.rename({"Node":"Parada","PosxNode":"X", "PosyNode":"Y", "Name":"Nombre"}, axis=1)

In [16]:
#Cambiamos la , por el . en la columna de X. Pasamos a tipo float X e Y
df_final["X"]=df_final["X"].str.replace(",",".")
df_final["X"].astype(float)
df_final["Y"].astype(float)

0        4480432.0
1        4480271.0
2        4479857.0
3        4479467.0
4        4479245.0
           ...    
10708    4481768.0
10709    4475811.0
10710    4475811.0
10711    4475803.0
10712    4475803.0
Name: Y, Length: 10713, dtype: float64

In [17]:
#Funcion para pasar las coordenadas x e y a latutud y longitud
def xy_lonlat(lista):
    x=lista[0]
    y=lista[1]
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=30, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return [lonlat[1], lonlat[0]]

In [18]:
#Creamos una lista con las coordenadas x e y pasadas a latitud y longitud
lista_long_lat=[xy_lonlat(i) for i in df_final[["X","Y"]].values.tolist()]

In [19]:
#Agregamos una columna con las latitudes y otra con las longitudes
df_final["Latitud"]=[i[0] for i in lista_long_lat]

df_final["Longitud"]=[i[1] for i in lista_long_lat]

In [20]:
#Quitamos las paradas que no estan en las lineas de la tabla anterior
df_final=df_final[df_final["Linea"].isin(df_lineas["Linea"].to_list())].reset_index().drop("index", axis=1)

In [21]:
# Creamos el id de la parada (que no es unico por parada si no por parada y linea/sentido realmente)
df_final.index=df_final.index+1
df_final=df_final.reset_index().rename({"index":"Id_Parada"}, axis=1)

In [22]:
# Creamos una columna para el id de la linea pero todo con ceros
df_final["Id_Linea"]=0

In [23]:
# Ponemos el id de la linea correspondiente
for id_linea, linea in df_lineas[["Id_Linea","Linea"]].values.tolist():
    df_final.loc[df_final["Linea"]==linea, "Id_Linea"]=id_linea

df_final.head()

,Id_Parada,Parada,Linea,Sentido,X,Y,Nombre,Latitud,Longitud,Id_Linea
0,1,1,161,Ida,433743.5,4480432,AV.VALDEMARIN-ALTAIR,40.471927,-3.781607,161
1,2,2,161,Ida,433481.3,4480271,AV.VALDEMARIN-LA SALLE,40.470456,-3.784683,161
2,3,3,161,Ida,433656,4479857,BLANCA DE CASTILLA-Cº DE LA ZARZUELA,40.466740,-3.782579,161
3,4,4,161,Ida,433632,4479467,PLEYADES-ANA TERESA,40.463225,-3.782821,161
4,5,5,161,Ida,433613.6,4479245,PLEYADES-AV.OSA MAYOR,40.461224,-3.783015,161


In [24]:
# Ordenamos los datos como queremos
df_final=df_final[["Id_Parada", "Parada", "Nombre", "Id_Linea", "Sentido", "Latitud", "Longitud"]]
df_final.head()

,Id_Parada,Parada,Nombre,Id_Linea,Sentido,Latitud,Longitud
0,1,1,AV.VALDEMARIN-ALTAIR,161,Ida,40.471927,-3.781607
1,2,2,AV.VALDEMARIN-LA SALLE,161,Ida,40.470456,-3.784683
2,3,3,BLANCA DE CASTILLA-Cº DE LA ZARZUELA,161,Ida,40.466740,-3.782579
3,4,4,PLEYADES-ANA TERESA,161,Ida,40.463225,-3.782821
4,5,5,PLEYADES-AV.OSA MAYOR,161,Ida,40.461224,-3.783015


In [25]:
# Pasamos los datos a csv
df_final.to_csv("paradas.csv", index=False)